# RANSAC-Experimente
Versuche, nach dem Voxelfilter mit RANSAC in die Schienen-Kandidatenpunkte innerhalb einer Nachbarschaft mit 1 m Radius eine Gerade zu fitten und inlier/outlier zu trennen. Wird so u.a. bei Kononen et al. (2024) gemacht.

RANSAC in 2D ohne z-Werte, in die outlier der ersten Gerade wird noch eine zweite gefittet (da bei Weichen zwei Schienen innerhalb des Radius liegen können).

Die Ergebnisse sind kaum brauchbar, weil RANSAC die Gerade in die Muster der Scanlinien fittet, statt der Schienenrichtung zu folgen. Dies ist bei größerem Schwellenwert weniger ausgeprägt, aber dann können Zunge/Backe bei Weichen nicht mehr getrennt werden. Daher wurde dieser Ansatz verworfen (Abschn. 5.5.5).

In [27]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.spatial import KDTree
import os
import json
import pyvista as pv
from sklearn.linear_model import RANSACRegressor

from interessant import * # Bei Änderungen Kernel neu starten

In [28]:
run = run24
#run = run14
# filename = interessant['OLA gleiche Höhe wie Gleis']

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A (in run24 Achszähler): 6; 
# Viele Gleise: 33; Anfang Weiche: 34; OLA gleiche H: 35; Y: 37
key = list(interessant.keys())[34] 
filename = interessant[key]
print(key, filename)

filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

Anfang Weiche 4481275_5357000.copc.laz


## Parameter

In [29]:
thresh = 8  # z.B. 5 oder 8
majority_tresh  = 0.5 # Erster Durchgang 0.3, bei "Gleis hohe Intensität" gibt 0.5 ein viel besseres Ergebnis

voxel_size = 1.0

voxel_size = 25 / 30
print("Voxel size:", voxel_size)

minimum_points = 50 # Erste Versuche mit 100, aber viel schwarz bei abseits liegenden Gleisen. 50 ist besser.
minimum_in_hood = 10
linearity_tresh = 0.98

intensity_threshold = 14500
downsample_radius = 0.3
neighborhood_radius = 0.5

Voxel size: 0.8333333333333334


## Voxelfilter
(erste, nicht optimierte Version)

In [30]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]

In [31]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()

points['Classification'] = 0 # Unclassified
RAIL = 20

maxp = xyz.max(axis=0)
minp = xyz.min(axis=0)

In [32]:
voxels = xyz.copy()
voxels[:, :2] = ((xyz[:, :2] - minp[:2]) // voxel_size).astype(int)

In [33]:
# Anzahl der Voxel checken
np.ceil((maxp[:2] - minp[:2]) / voxel_size).astype(int)

array([30, 30])

In [34]:
from collections import defaultdict
voxel_dict = defaultdict(list)
index_dict = defaultdict(list)

# Füllen des Dictionaries
for idx, (point, voxel) in enumerate(zip(xyz, voxels)):
    voxel_key = tuple(voxel[:2])
    voxel_dict[voxel_key].append(point[2])
    index_dict[voxel_key].append(idx)

In [35]:
for key, z_values in voxel_dict.items():
    
    # Threshold on number of points in voxel
    if len(z_values) < minimum_points:
        continue

    indices = np.array(index_dict[key])
    z_values = np.array(z_values)
    ground_level = np.percentile(z_values, 10) # 10% Percentile
    # Check that there are almost no points 0.5 to 4.5 m above the ground
    # But allow for some noise
    # thresh = 3 # Der einfachheit halber oben
    count = ((z_values > ground_level + 0.5) & (z_values < ground_level + 4.5)).sum()

    if count <= thresh:
        # Look for points within 0.5 m above ground and get 98% percentile ODER 99.5
        mask = (z_values > ground_level) & (z_values < ground_level + 0.5)
        try:
            candidates_top = np.percentile(z_values[mask], 99.5)
        except IndexError:
            # Fails if there are no points in the masked array
            continue

        # Oude Elberink require the height difference > 0.1 m
        # And mark only the points 10 cm below the top as rail point candidates
        if candidates_top - ground_level > 0.1:
            mask = (z_values > candidates_top - 0.1) & (z_values < candidates_top + 0.05)

            # Also make sure these are only a minority of the points (otherwise it's a slope)
            if mask.sum() < majority_tresh * len(z_values):  # z.B. 0.3
                points['Classification'][indices[mask]] = RAIL


In [36]:
candidates = points[points["Classification"] == RAIL]
candidates.shape

(154816,)

## View Settings

In [37]:
# Viewsettings mit strg + c kopieren und hier einfügen

viewsettings = '''
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 11.999975427985191, 11.99998692702502, 13.124079998226534 ],
			"boundingbox_min" : [ -13.000024572014809, -13.00001307297498, -3.9965200017734333 ],
			"field_of_view" : 60.0,
			"front" : [ -0.20468464372193082, -0.82045900926496551, 0.53380821531742795 ],
			"lookat" : [ -2.1145501200370735, -2.6052610037108783, 1.4494799802055294 ],
			"up" : [ 0.19010212482081987, 0.50164960558000959, 0.84392467398461002 ],
			"zoom" : 0.55999999999999983
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

'''

viewsettings = json.loads(viewsettings)

front = viewsettings["trajectory"][0]["front"]
lookat = viewsettings["trajectory"][0]["lookat"]
up = viewsettings["trajectory"][0]["up"]
zoom = viewsettings["trajectory"][0]["zoom"]

## Kandidaten und Saatpunkte

Noise Filter auf dem Ergebnis des Voxelfilters

In [38]:
# filters.outlier sets Classification to 7, filters.range removes the points with Classification 7

noise_filter = pdal.Filter("filters.outlier", method="statistical", mean_k=10, multiplier=2.0).pipeline(candidates) | pdal.Filter("filters.range", limits="Classification![7:7]")
print(noise_filter.toJSON())
noise_filter.execute()
candidates = noise_filter.arrays[0]
candidates.shape 

[{"type": "filters.outlier", "method": "statistical", "mean_k": 10, "multiplier": 2.0, "tag": "filters_outlier1"}, {"type": "filters.range", "limits": "Classification![7:7]", "tag": "filters_range1"}]


(151900,)

In [39]:
xyz = np.vstack((candidates['X'], candidates['Y'], candidates['Z'])).transpose()
offset = xyz.mean(axis=0).round() 
xyz -= offset

Saatpunkte

In [40]:
low_intensity = candidates[candidates["Intensity"] < intensity_threshold]
low_intensity.shape

(96434,)

In [41]:
# Downsample with poisson sampling

downsampling_pipeline = pdal.Filter("filters.sample", radius=downsample_radius).pipeline(low_intensity)
downsampling_pipeline.execute()
seed_points = downsampling_pipeline.arrays[0]
seed_points.shape 

(614,)

In [42]:
xyz_seed = np.vstack((seed_points['X'], seed_points['Y'], seed_points['Z'])).transpose()
xyz_seed -= offset

k-d-Baum

In [43]:
# k-D tree with all candidate points
tree = KDTree(xyz)  

In [44]:
# indices: ndarray (dtype object) with a list of indices for each seed point
indices = tree.query_ball_point(xyz_seed, r=neighborhood_radius)

In [45]:
seed_point_count = xyz_seed.shape[0]

## Funktionen

In [46]:
def pca(cloud):
    """Use PCA to get einvalues and eigenvectors of a point cloud"""
    mean = np.mean(cloud, axis=0)
    centered = cloud - mean
    cov_matrix = np.cov(centered, rowvar=False) # row variance nicht berechnen
    eigenvals, eigenvecs = np.linalg.eig(cov_matrix)
    sorted_indices = np.argsort(eigenvals)[::-1]
    sorted_eigenvals = eigenvals[sorted_indices]
    sorted_eigenvecs = eigenvecs[:,sorted_indices]
    return sorted_eigenvals, sorted_eigenvecs

def linearity(eigenvals):
    """Calculate the linearity of a point cloud"""
    return (eigenvals[0] - eigenvals[1]) / eigenvals[0]


In [47]:
def fit_line_ransac(points, threshold=0.1, min_samples=20):
    """Fits a line to the given 2D points using RANSAC."""
    if len(points) < min_samples:
        return None, None  # Not enough points

    ransac = RANSACRegressor(residual_threshold=threshold)
    ransac.fit(points[:, [0]], points[:, 1])  # X -> Y mapping
    
    inlier_mask = ransac.inlier_mask_
    a, b = ransac.estimator_.coef_[0], ransac.estimator_.intercept_
    
    return (a, b), inlier_mask  # Line equation and inliers

## RANSAC auf Nachbarschaft um einen Saatpunkt
Der Index eines Saatpunkts muss gewählt werden. Verschiedene Schwellenwerte können ausprobiert werden.


Erste RANSAC-Linie: rot, zweite, in deren outlier gefittete RANSAC-Linie: grün.

In [50]:
i = 0 # Index eines Saatpunkts wählen

# RANSAC-Schwellenwert
threshold = 0.05 # 0.05 0.1, 0.2, 0.3

fit_second_line = True

hood = xyz[indices[i]]
if hood.shape[0] < minimum_in_hood:   
    raise ValueError("Not enough points in neighborhood") 
print(f"Seed point {i} has {hood.shape[0]} points in its neighborhood")

# Linearity der Nachbarschaft 
eigenvals, eigenvecs = pca(hood)
print("Linearity:", linearity(eigenvals))
print("Eigenvals", eigenvals)

# RANSAC in 2D ohne z-Werte
hood_2d = hood[:, :2]

# Erste RANSAC-Linie
model1, inliers1 = fit_line_ransac(hood_2d, threshold)


if fit_second_line and model1 is not None:
    # ggf. zweite RANSAC-Linie in die outlier der ersten Linie fitten
    print("Model 1", model1, "Inliers", inliers1.sum())
    rest = hood_2d[~inliers1]
    model2, inliers2 = fit_line_ransac(rest, threshold)
else:
    model2 = None


# Visualisierung

colors = np.ones((hood.shape[0], 3), dtype=np.float64) 
colors = colors * 0.5 # Gray

if model1 is not None:
    colors[inliers1] = [1, 0, 0]  # Red
else:
    print("No model")
if model2 is not None:
    print("Model 2", model2, "Inliers", inliers2.sum())
    # Inliers2 has the shape of the remains of RANSAC pass 1
    # Map it back to the original indices

    inliers2_full = np.zeros(hood.shape[0], dtype=bool)
    inliers2_full[~inliers1] = inliers2
    colors[inliers2_full] = [0, 1, 0]  # Green


hood_pcd = o3d.geometry.PointCloud()
hood_pcd.points = o3d.utility.Vector3dVector(hood)
hood_pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([hood_pcd])


Seed point 0 has 452 points in its neighborhood
Linearity: 0.9864199962164693
Eigenvals [0.08305251 0.00112785 0.0003418 ]
Model 1 (np.float64(0.09846303696418181), np.float64(4.719894834146913)) Inliers 408
Model 2 (np.float64(0.18695045200653862), np.float64(4.097253315778029)) Inliers 41
